# Using SQL to Answer Bussiness Questions

The database used on this project is called Chinook.db and it contains tables of music sales with all the information about artist.

The database can be found [here]().

The main purpose of this project is to answer bussiness questions using SQL, so the questions will appear and the process to answer them will be under the question.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Finding the top 3 sales genres in USA

In [3]:
%%sql
WITH altra AS (
            SELECT ge.*, i.billing_country country
            FROM genre ge
            LEFT JOIN track tr ON tr.genre_id= ge.genre_id
            LEFT JOIN invoice_line il ON il.track_id=tr.track_id
            LEFT JOIN invoice i ON i.invoice_id=il.invoice_id),

     coun AS (
           SELECT *
             FROM altra 
            WHERE country='USA'),
    gen AS (
            SELECT *, COUNT(name) track_sold
            FROM coun
            GROUP BY genre_id),
    total AS (
            SELECT
            SUM(track_sold) total
            FROM gen)

SELECT *, ROUND(CAST(track_sold AS FLOAT)/1051*100,4) tracks_sold_per
FROM gen
ORDER BY track_sold DESC
/*1051*/

Done.


genre_id,name,country,track_sold,tracks_sold_per
1,Rock,USA,561,53.3777
4,Alternative & Punk,USA,130,12.3692
3,Metal,USA,124,11.7983
14,R&B/Soul,USA,53,5.0428
6,Blues,USA,36,3.4253
23,Alternative,USA,35,3.3302
7,Latin,USA,22,2.0932
9,Pop,USA,22,2.0932
17,Hip Hop/Rap,USA,20,1.9029
2,Jazz,USA,14,1.3321


Poner aqui que es lo que dice lo de arriba y la recomendacion que le haria a los artistas de los albums que voy a subir.

### Total dolar amount of sales assigned to each sales support agent

In [4]:
%%sql
SELECT 
       em.employee_id,
       em.first_name||' '||em.last_name Name,
       SUM(i.total) sales,
       em.hire_date,
        COUNT(cu.customer_id) clients
FROM employee em
LEFT JOIN customer cu ON cu.support_rep_id=em.employee_id
LEFT JOIN invoice i ON i.customer_id=cu.customer_id
WHERE em.title='Sales Support Agent'
GROUP BY 1
ORDER BY 3 DESC;

Done.


employee_id,Name,sales,hire_date,clients
3,Jane Peacock,1731.510000000004,2017-04-01 00:00:00,212
4,Margaret Park,1584.0000000000032,2017-05-03 00:00:00,214
5,Steve Johnson,1393.9200000000028,2017-10-17 00:00:00,188


We have a total of 3 people working as a Sales Support agent. The one who has sale the most is Jane Peacock and the less one is Steve Jhonson, but these amount of sales are not inside the same timeframe. 

Jane Peacock is the Sales Support Manager employee with the most time working in the area with a difference of 5 months against Steve Jhonson, the on quo has less sales, so we can not compare theur sales.

We can consider the amount od customers each Sales Agent has in order to see if that could affect each employee. An watching the column `clients`, we can see that inn fact maybe the amount of clients is not giving `Steve Johnson` more sales margin.

### Purchases from different countries

In [5]:
%%sql
WITH tabla1 AS (
             SELECT country,
             COUNT(customer_id) Clients,
             CASE 
                WHEN
                    COUNT(customer_id)=1 THEN 'OTHER'
                    ELSE country
                    END AS Countries
            FROM customer   
            GROUP BY 1),

tabla2 AS (
             SELECT Countries, Clients,
             CASE
                 WHEN 
                    Countries='OTHER' THEN 1
                    ELSE 0
                    END AS SORT
            FROM tabla1),

tabla3 AS (
             SELECT c.country, SUM(total) total
               FROM customer as c
             LEFT JOIN invoice i ON i.customer_id=c.customer_id
             GROUP BY 1),
    
tabla4 AS (
            SELECT Countries, Clients, total,
            CASE
                WHEN 
                    Countries='OTHER' THEN 1
                    ELSE 0
                    END AS SORT
            FROM tabla1
            LEFT JOIN tabla3 ON tabla3.country=tabla1.country
    )


SELECT Countries, 
        SUM(Clients) Customers, 
        ROUND(SUM(total),2) Sales,
        ROUND(SUM(total)/SUM(Clients),2) AVG_Sales
FROM tabla4
GROUP BY Countries
ORDER BY SORT ASC, 2 DESC;

Done.


Countries,Customers,Sales,AVG_Sales
USA,13,1040.49,80.04
Canada,8,535.59,66.95
Brazil,5,427.68,85.54
France,5,389.07,77.81
Germany,4,334.62,83.65
United Kingdom,3,245.52,81.84
Czech Republic,2,273.24,136.62
India,2,183.15,91.57
Portugal,2,185.13,92.56
OTHER,15,1094.94,73.0


Dar un consejo y falta dividir ventas entre el numero de tickets

### Albums vs Single tracks

In [6]:
%%sql

WITH albums_tracks AS(
                SELECT album.album_id,
                COUNT(track_id) Tracks_Album
    
                FROM album 
                LEFT JOIN track ON track.album_id=album.album_id
                GROUP BY 1),

tracks_invoice AS (
                SELECT invoice_id, 
                COUNT(track.track_id) Tracks,
                MIN(track.album_id) Album
                FROM invoice_line
                LEFT JOIN track ON track.track_id=invoice_line.track_id
                GROUP BY 1),

ita AS (
                SELECT *
                FROM tracks_invoice
                LEFT JOIN albums_tracks ON tracks_invoice.Album=albums_tracks.album_id),

t AS (
                SELECT *,
                    CASE 
                        WHEN Tracks=Tracks_Album THEN 'Album'
                        ELSE 'Single_Tracks'
                        END AS WHAT
                FROM ita),

t2 AS (
                SELECT 
                COUNT(*) Invoices,
                                (
                                    SELECT COUNT(*) 
                                    FROM t
                                    WHERE WHAT='Album') Albums_purchased,
                                (
                                    SELECT COUNT(*) 
                                    FROM t
                                    WHERE WHAT='Single_Tracks') Trakcs_Purchased
    
                FROM t),

prueba AS (
                SELECT WHAT, 
                COUNT(invoice_id) Sales
                FROM t
                GROUP BY WHAT),

total AS (
                SELECT COUNT(*) Total
                FROM t)

    
/*614*/
SELECT *,
        ROUND(CAST(Sales AS FLOAT)/614*100,2) || '%' percentage
FROm prueba;

Done.


WHAT,Sales,percentage
Album,135,21.99%
Single_Tracks,479,78.01%


22% of the invoices are from Albums, while 78% from single tracks, I would not recommend to stop buying Albums from the record companies because they represent,approximatly,  one fifth of the revenue. 

### Which artist is used in the most playlist

In [7]:
%%sql
WITH tr_al AS (
                SELECT ar.artist_id,
                        ar.name,
                        tr.track_id
                FROM artist ar
                LEFT JOIN album al ON al.artist_id=ar.artist_id
                LEFT JOIN track tr ON tr.album_id=al.album_id),
     play AS (
                 SELECT playlist_id,
                         tr_al.*
                         FROM playlist_track pt
                        LEFT JOIN tr_al ON tr_al.track_id=pt.track_id),
    art AS (
                SELECT playlist_id,
                        name,
                        COUNT(name) number
                FROM play
                GROUP BY 1,2             
    ),
    
    final AS (
                SELECT playlist_id,
                        name,
                        MAX(number) 
                FROM art
                GROUP BY playlist_id
    
    )
 
SELECT *
FROM final;

Done.


playlist_id,name,MAX(number)
1,Iron Maiden,213
3,Lost,92
5,Iron Maiden,84
8,Iron Maiden,213
9,Audioslave,1
10,Lost,92
11,Tim Maia,7
12,Berliner Philharmoniker & Herbert Von Karajan,3
13,"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",1
14,Berliner Philharmoniker & Herbert Von Karajan,2


In [8]:
%%sql
/*Here is the code to check each playlist to be sure everything is ok*/
WITH tr_al AS (
                SELECT ar.artist_id,
                        ar.name,
                        tr.track_id
                FROM artist ar
                LEFT JOIN album al ON al.artist_id=ar.artist_id
                LEFT JOIN track tr ON tr.album_id=al.album_id),
     play AS (
                 SELECT playlist_id,
                         tr_al.*
                         FROM playlist_track pt
                        LEFT JOIN tr_al ON tr_al.track_id=pt.track_id)
    
SELECT *
FROM play
WHERE playlist_id IS 18;

Done.


playlist_id,artist_id,name,track_id
18,68,Miles Davis,597


### How many tracks have been purchased vs not purchased?

In [29]:
%%sql
WITH compradas AS (
                    SELECT DISTINCT(track_id)
                    FROM invoice_line
                    ),
canciones AS (
                    SELECT track_id
                    FROM track
                    ),

No_compradas AS (
                SELECT *
                FROM canciones
                EXCEPT
                SELECT *
                FROM compradas
)

SELECT 
        COUNT(*) Songs,
        ( SELECT COUNT(*) FROM compradas) Purchased,
        (SELECT COUNT(*) FROM No_Compradas) No_Purchased
FROM canciones

Done.


Songs,Purchased,No_Purchased
3503,1806,1697
